<a href="https://colab.research.google.com/github/gackouhamady/TP1_Parquet_Avro/blob/main/Parquet_Avro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>